<a href="https://colab.research.google.com/github/Niklas123456789/DM/blob/main/task2/generate_kernels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#preinstalled version of pytorch has to be the same as the pre-compiled versions of the pytorch-geometric packages that we download later on.
#versions might change quickly, so if you get a strange error later on, check the torch version of Google colab later on as follows:

import torch
torch.__version__

'1.7.0+cu101'

In [2]:
# Script to generate variations of the kernels yourself
# https://ucloud.univie.ac.at/index.php/s/E3YKph0jkpbw8TN


# #Download the TUDataset Repository with
!git clone https://github.com/chrsmrrs/tudataset.git
# #move this script to tudataset/tud_benchmark

# #Install pytorch geometric: https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
# #Here is the gpu cuda installation, for the cpu version replace cu102 with cpu
%pip --no-cache-dir install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-geometric

%pip --no-cache-dir install networkit

%pip --no-cache-dir install pybind11
!sudo apt-get install libeigen3-dev

%cd ..
%cd /content/tudataset/tud_benchmark/kernel_baselines/
! ls
! g++ -I /usr/include/eigen3 -O3 -shared -std=c++11 -fPIC `python3 -m pybind11 --includes`  kernel_baselines.cpp src/*cpp -o ../kernel_baselines`python3-config --extension-suffix`
%cd ..

!ls -al /usr/local/cuda

fatal: destination path 'tudataset' already exists and is not an empty directory.
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 11.9MB 20.0MB/s 
  Found existing installation: torch-scatter 2.0.5
    Uninstalling torch-scatter-2.0.5:
      Successfully uninstalled torch-scatter-2.0.5
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 24.3MB 101.7MB/s 
  Found existing installation: torch-sparse 0.6.8
    Uninstalling torch-sparse-0.6.8:
      Successfully uninstalled torch-sparse-0.6.8
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 21.5MB 1.1MB/s 
  Found existing installation: torch-cluster 1.5.8
    Uninstalling torch-cluster-1.5.8:
      Successfully uninstalled torch-cluster-1.5.8
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 6.4MB 21.3MB/s

In [19]:
import os
import torch
import numpy as np
import kernel_baselines as kb
import auxiliarymethods
import auxiliarymethods.auxiliary_methods as aux
import auxiliarymethods.kernel_evaluation as ke
import auxiliarymethods.datasets as dp
from scipy.sparse import save_npz

import networkit as nk
import networkx as nx
from auxiliarymethods.reader import tud_to_networkx

In [4]:
def setup_directory(dir_name, verbose=False):
    """
    Setup directory in case it does not exist
    Parameters:
    -------------
    dir_name: str, path + name to directory
    verbose: bool, indicates whether directory creation should be printed or not.
    """
    if not os.path.exists(dir_name):
        try:
            os.makedirs(dir_name)
            if verbose:
                print("Created Directory: {}".format(dir_name))
        except Exception as e:
            raise RuntimeError(
                "Could not create directory: {}\n {}".format(dir_name, e))


In [5]:
use_edge_labels = False
for USE_LABELS in [True, False]:# Except IMDB-BINARY
    break
    for dataset, use_labels in [["IMDB-BINARY", False],["MSRC_21",USE_LABELS], ["NCI1", USE_LABELS], ["ENZYMES", USE_LABELS]]:
        if use_labels:
            base_path = os.path.join("kernels","node_labels")
        else:
            base_path = os.path.join("kernels","without_labels")
        setup_directory(base_path)
        print("Start processing data set ", dataset)
        # Download dataset.
        classes = dp.get_dataset(dataset)
        # *Weisfeihler-Lehman*
        print("Start computing Weisfeihler-Lehman gram matrix and vector representations")
        iterations = 6
        #0 taking just the nodelabels themselves into account; 
        #1 considers nearest-neighbours, 2 one layer deeper and so on
        for i in range(1, iterations):
            print("Start iteration ", i)
            #Gram Matrix for the Weisfeiler-Lehman subtree kernel
            gram_matrix_wl = kb.compute_wl_1_dense(dataset, i, use_labels, use_edge_labels)
            np.savetxt(os.path.join(base_path,f"{dataset}_gram_matrix_wl{i}.csv"),
                    gram_matrix_wl,
                    delimiter=";")
            #Sparse Vectors for the Weisfeiler-Lehmann subtree kernel
            vectors_wl = kb.compute_wl_1_sparse(dataset, i, use_labels, use_edge_labels)
            save_npz(os.path.join(base_path,f"{dataset}_vectors_wl{i}.npz"),
                    vectors_wl, compressed=True)

        # *Graphlet kernel*
        print("Start computing Graphlet gram matrix")

        #Gram Matrix for the Graphlet kernel
        gram_matrix_graphlet= kb.compute_graphlet_dense(dataset, use_labels, use_edge_labels)
        np.savetxt(os.path.join(base_path,f"{dataset}_gram_matrix_graphlet.csv"),
                gram_matrix_graphlet,
                delimiter=";")

        print("Start computing Graphlet vector representation")
        #Sparse Vectors for the Graphlet kernel
        vectors_graphlet = kb.compute_graphlet_sparse(dataset, use_labels, use_edge_labels)
        save_npz(os.path.join(base_path,f"{dataset}_vectors_graphlet.npz"),
                vectors_graphlet, compressed=True)


        print("Start computing Shortest path gram matrix")

        #Gram Matrix for the Shortest path kernel
        gram_matrix_shortestpath = kb.compute_shortestpath_dense(dataset, use_labels)
        np.savetxt(os.path.join(base_path,f"{dataset}_gram_matrix_shortestpath.csv"),
                gram_matrix_shortestpath,
                delimiter=";")

        print("Start computing Shortest path vector representation")

        #Sparse Vectors for the Shortest path kernel
        vectors_shortestpath = kb.compute_shortestpath_sparse(dataset, use_labels)
        save_npz(os.path.join(base_path,f"{dataset}_vectors_shortestpath.npz"),
                vectors_shortestpath, compressed=True)



In [6]:
def density_list(G):
  return [2*g.numberOfEdges()/(g.numberOfNodes() * (g.numberOfNodes()-1)) for g in G]

In [7]:
dataset = "IMDB-BINARY"
classes = dp.get_dataset(dataset)
G = tud_to_networkx(dataset)
nkG = [nk.nxadapter.nx2nk(g) for g in G]

In [20]:
all_matrices = []
num_reps = 10
use_labels, use_edge_labels = False, False

# 1-WL kernel, number of iterations in [1:6].
for i in range(1, 6):
    gm = kb.compute_wl_1_dense(dataset, i, use_labels, use_edge_labels)
    # Cosine normalization.
    gm = aux.normalize_gram_matrix(gm)
    all_matrices.append(gm)

# print(accuracy, std_10, std_100) == 37.63333333333334 1.8315142490421534 6.490077897152784
accuracy, std_10, std_100 = ke.kernel_svm_evaluation(all_matrices, classes, num_repetitions=num_reps, all_std=True)

In [21]:
print(accuracy, std_10, std_100)

37.63333333333334 1.8315142490421534 6.490077897152784
